In [1]:
REQUEST_BODY = {
    "query": {
        "type": "all",
        "allQueries": [
            {
                "boost": 1,
                "field": "body.publishDate.date",
                "from": 0, # FROM
                "to": 0, # TO
                "includeFrom": True,
                "includeTo": True,
                "type": "range"
            },
            {
                "type": "nested",
                "field": "metaData.authors",
                "boost": 1.0,
                "query": 
                {
                    "type": "term",
                    "field": "id",
                    "value": "22c4c232-5f85-4e6c-b083-663b56976fe2" #Influencer UUID
                }
            },
            {
                "type": "term",
                "field": "metaData.source.informationType",
                "value": "news" #For doc media type
            }
        ]
    },
    "viewRequests": {
        "count": {
            "type": "count"
        }#,
        #"list": {
        #    "type": "resultList",
        #    "fields": [
        #        "id"
        #    ],
        #    "size": 100,
        #    "start": 0
        #}
    }
}

In [7]:
import pandas as pd
from operator import itemgetter

DATA_PATH = "contacts_10_+articles.csv"
influencers_df = pd.read_csv(DATA_PATH)
print(len(influencers_df))

34


In [8]:
def create_timestamp(time_tuple):
    return int(time.mktime(time_tuple))*1000

In [9]:
import requests
import copy
import json
import time

def documentcount_for_date(uuid, from_, to_, search_url):
    try:
        body = copy.deepcopy(REQUEST_BODY)
        body["query"]["allQueries"][1]["query"]["value"] = uuid
        body["query"]["allQueries"][0]["from"] = create_timestamp(from_)
        body["query"]["allQueries"][0]["to"] = create_timestamp(to_)
        #print(json.dumps(body, indent=2))
        result = requests.request("POST", url=search_url, data=json.dumps(body))
        ret = result.json()["views"]["count"]["totalCount"]
    except Exception as e:
        print("Exception:", e)
        ret = 0
    #print("{}: {}".format(uuid,ret))
    return ret

def documentcount_in_2015_for_uuid(uuid, search_url):
    from_2015 = (2015,1,1,0,0,0,0,0,0)
    to_2016 = (2016,1,1,0,0,0,0,0,0)
    return documentcount_for_date(uuid, from_2015, to_2016, search_url)


#documentcount_in_2015_for_uuid("b31b246f-46e4-4532-92ae-26b29bdf3966")

Sorting "influencer id" based on deep update

In [10]:
#influencers_df_sorted.uuid[0:2].map(documentcount_in_2015_for_uuid)

SEARCH_URL_STAGING = "http://fh-stage-c.meltwater.net/searchservice/"
SEARCH_URL_PRODUCTION = "http://mag-fh-searchservice.osl.basefarm.net:8080/searchservice/"
new_columns = ["UUID", "doc_staging","doc_production"]
influencers_df_doc = pd.DataFrame(columns=new_columns)
for index, row in influencers_df.iterrows():
    num_doc_staging = documentcount_in_2015_for_uuid(row["UUID"], SEARCH_URL_STAGING)
    num_doc_production = documentcount_in_2015_for_uuid(row["UUID"], SEARCH_URL_PRODUCTION)
    #print (num_doc_production)
    influencers_df_doc.loc[len(influencers_df_doc)] = [row['UUID'], num_doc_staging, num_doc_production]
influencers_df_doc

,UUID,doc_staging,doc_production
0,e540d28c-fc0a-4604-86ab-739f046be76d,41,52
1,cde1f7c0-624a-4d3b-80ea-1bfcb31bcec5,27,29
2,401c5983-35f7-4f27-9c34-aa0f8a0074a5,367,458
3,07961b32-a6b8-430c-ab61-eb41b0b48726,409,1043
4,534a3d4e-b68c-4122-90c3-d7681f3d9260,495,606
5,f38ac2e3-4f5f-4073-b3b9-8367ffeaa336,8,8
6,e9621df6-db86-4d35-b3a9-32d01d970794,32,48
7,c6ffde39-86cf-4612-8875-e77fdd971f0b,0,0
8,4ebb35eb-cb73-47d7-9534-678e3b6aa8a1,20,28
9,82bd917e-5537-4227-817b-d149893bf018,47,65


In [11]:
merged_df = pd.merge(influencers_df, influencers_df_doc)
merged_df

,UUID,ID,Name,Outlet Type,Role,doc_staging,doc_production
0,e540d28c-fc0a-4604-86ab-739f046be76d,582493,Kelly Bartnick,Television Station,Reporter,41,52
1,cde1f7c0-624a-4d3b-80ea-1bfcb31bcec5,173358,Garrett Gordon,Television Station,"Reporter, Host",27,29
2,401c5983-35f7-4f27-9c34-aa0f8a0074a5,131863,Brent Martin,Radio Network,News Director,367,458
3,07961b32-a6b8-430c-ab61-eb41b0b48726,601891,Nell Minow,Online News Source,Movie Critic,409,1043
4,534a3d4e-b68c-4122-90c3-d7681f3d9260,664009,Dominique Mosbergen,Online News Source,Senior Writer,495,606
5,f38ac2e3-4f5f-4073-b3b9-8367ffeaa336,563311,Craig Offman,Newspaper,Staff Writer,8,8
6,e9621df6-db86-4d35-b3a9-32d01d970794,316698,Josh O'Kane,Newspaper,Reporter,32,48
7,c6ffde39-86cf-4612-8875-e77fdd971f0b,1100260,Clare O'Hara,Newspaper,Reporter,0,0
8,4ebb35eb-cb73-47d7-9534-678e3b6aa8a1,54676,Paul Basken,Online News Source,Reporter,20,28
9,82bd917e-5537-4227-817b-d149893bf018,18674,Cary Blake,Online News Source,Editor,47,65


In [7]:
merged_df.to_csv("contact_more_than_10_doc_stats.csv")